In [5]:
from pathlib import Path
import tifffile
import numpy as np
from PIL import Image

input_dir = Path("E:/Cellula Tech/Computer Vision/Task 4/data/train/images")
output_dir = Path("E:/Cellula Tech/Computer Vision/Task 4/data/train/images_png")
output_dir.mkdir(parents=True, exist_ok=True)

for img_path in input_dir.glob("*.tif"):
    arr = tifffile.imread(img_path)

    # If it's single-channel (grayscale), expand to 3 channels
    if arr.ndim == 2:
        arr = np.stack([arr] * 3, axis=-1)

    # If it has >= 4 bands, select bands [3,2,1] (assuming indexing starts at 0 → [2,1,0])
    if arr.shape[-1] >= 3:
        arr = arr[:, :, [3, 2, 1]] if arr.shape[-1] > 3 else arr[:, :, :3]

    # Normalize if not uint8
    if arr.dtype != np.uint8:
        arr = (255 * (arr.astype(np.float32) / arr.max())).astype(np.uint8)

    # Save as PNG instead of JPG
    img = Image.fromarray(arr)
    img.save(output_dir / f"{img_path.stem}.png")


In [3]:
from pathlib import Path

labels = Path("E:/Cellula Tech/Computer Vision/Task 4/data/train/labels")
for f in labels.glob("*.txt"):
    with open(f, "r") as file:
        lines = file.readlines()
        if lines:
            print(f"{f.name} ✅ {len(lines)} objects")
        else:
            print(f"{f.name} ❌ empty")


0.txt ✅ 44 objects
1.txt ✅ 4 objects
10.txt ✅ 14 objects
101.txt ✅ 2 objects
102.txt ✅ 22 objects
103.txt ❌ empty
104.txt ✅ 6 objects
105.txt ✅ 11 objects
106.txt ❌ empty
107.txt ❌ empty
108.txt ✅ 12 objects
110.txt ✅ 25 objects
113.txt ✅ 10 objects
114.txt ✅ 39 objects
115.txt ❌ empty
117.txt ✅ 31 objects
12.txt ✅ 8 objects
121.txt ✅ 8 objects
122.txt ❌ empty
123.txt ✅ 14 objects
124.txt ✅ 13 objects
125.txt ✅ 26 objects
126.txt ✅ 13 objects
127.txt ✅ 13 objects
128.txt ✅ 11 objects
129.txt ✅ 18 objects
130.txt ✅ 27 objects
131.txt ✅ 16 objects
132.txt ✅ 40 objects
133.txt ✅ 1 objects
135.txt ✅ 13 objects
136.txt ❌ empty
137.txt ✅ 25 objects
139.txt ✅ 22 objects
14.txt ❌ empty
141.txt ✅ 10 objects
143.txt ✅ 28 objects
144.txt ✅ 11 objects
146.txt ✅ 4 objects
147.txt ✅ 9 objects
148.txt ✅ 13 objects
149.txt ✅ 34 objects
150.txt ✅ 1 objects
152.txt ❌ empty
153.txt ✅ 14 objects
154.txt ✅ 19 objects
156.txt ✅ 39 objects
157.txt ✅ 9 objects
158.txt ✅ 8 objects
159.txt ✅ 7 objects
16.txt ✅ 

In [8]:
from pathlib import Path

root = Path("E:/Cellula Tech/Computer Vision/Task 4/data")

for split in ["train", "val"]:
    img_dir = root / split / "images"
    lbl_dir = root / split / "labels"

    print(f"\nChecking {split} set...")
    missing, empty = 0, 0
    for img_path in img_dir.glob("*.jpg"):
        lbl_path = lbl_dir / f"{img_path.stem}.txt"
        if not lbl_path.exists():
            print(f"❌ Missing label for {img_path.name}")
            missing += 1
        else:
            if lbl_path.stat().st_size == 0:
                print(f"⚠️ Empty label file: {lbl_path.name}")
                empty += 1
    print(f"Done. Missing: {missing}, Empty: {empty}")



Checking train set...
Done. Missing: 0, Empty: 0

Checking val set...
Done. Missing: 0, Empty: 0


In [11]:
from ultralytics import YOLO

# Path to your dataset.yaml
yaml_path = "E:/Cellula Tech/Computer Vision/Task 4/data/data.yaml"

# Load YOLOv8 segmentation model (pick size depending on GPU)
model = YOLO("yolov8n-seg.pt")  # yolov8n-seg, yolov8s-seg, yolov8m-seg, yolov8l-seg, yolov8x-seg

# Train
model.train(
    data=yaml_path,
    epochs=500,      # adjust as needed
    imgsz=640,       # training image size
    batch=8,         # adjust to fit GPU memory
    workers=2,       # dataloader workers
    device=0         # GPU id, or 'cpu' if no GPU
)


New https://pypi.org/project/ultralytics/8.3.203 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.198  Python-3.12.0 torch-2.6.0.dev20241112+cu121 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:/Cellula Tech/Computer Vision/Task 4/data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0,

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020B2B51B710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [17]:
# Load trained weights
model = YOLO("runs/segment/train/weights/best.pt")

# Run inference on a validation image
results = model.predict(
    source="E:/Cellula Tech/Computer Vision/Task 4/data/val/images/298.png",
    save=True,   # saves output with masks drawn
    show=True    # shows in window (remove if running on server)
)



image 1/1 E:\Cellula Tech\Computer Vision\Task 4\data\val\images\298.png: 640x640 (no detections), 24.4ms
Speed: 5.5ms preprocess, 24.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to E:\Cellula Tech\Computer Vision\Task 4\runs\segment\predict4
